Meta_Data_Reader

In [11]:
# Import necessary libraries
%matplotlib widget
%load_ext autoreload
%autoreload 2

import h5py
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Function to read and display metadata
def read_metadata(file_path):
    if not os.path.exists(file_path):
        return f"Error: File does not exist at {file_path}"
    
    try:
        with h5py.File(file_path, 'r') as f:
            metadata = {
                'Spot Size': f['metadata']['spot_size'][()],
                'Step Size (Å)': f['metadata']['step_size(m)'][()] / 1e-10,
                'Convergence Semi-Angle (rad)': f['metadata']['convergence_semi-angle(rad)'][()],
                'Camera Length (m)': f['metadata']['merlin_camera_length(m)'][()],
                'Magnification': f['metadata']['magnification'][()],
                'High Tension Value (V)': f['metadata']['ht_value(V)'][()],
                'Defocus (nm)': f['metadata']['defocus(nm)'][()]
            }
    except KeyError as e:
        return f"Metadata key not found: {e}"
    except OSError as e:
        return f"Could not open file: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"
    
    return metadata

# Function to handle the button click
def on_button_click(b):
    # Clear previous output
    clear_output(wait=True)
    display(file_path_text, load_button, output_area)
    
    # Read metadata from the entered file path
    file_path = file_path_text.value.strip()
    if not file_path:
        with output_area:
            clear_output()
            print("Error: File path cannot be empty. Please enter a valid file path.")
        return
    
    metadata = read_metadata(file_path)
    
    # Display metadata in output area
    with output_area:
        clear_output()
        if isinstance(metadata, dict):
            print("Extracted Metadata:")
            for key, value in metadata.items():
                print(f"{key}: {value}")
        else:
            print(metadata)  # Print error message if it's not a dictionary

# Create text input for file path and button to load metadata
file_path_text = widgets.Text(
    value='',
    placeholder='Enter the file path of the HDF file',
    description='File Path:',
    layout=widgets.Layout(width='80%')
)
load_button = widgets.Button(description="Load Metadata")

# Create an output area to show metadata
output_area = widgets.Output()

# Set up the button click event
load_button.on_click(on_button_click)

# Display the widgets
display(file_path_text, load_button, output_area)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Text(value='', description='File Path:', layout=Layout(width='80%'), placeholder='Enter the file path of the H…

Button(description='Load Metadata', style=ButtonStyle())

Output()

Probe Diamter calculator

In [12]:
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Label, Output
import math

# Function for Equation 1: Overlap %
def calculate_overlap_percent(step_size, probe_diameter):
    try:
        overlap = (1 - (step_size / probe_diameter)) * 100
        return f"Overlap %: {overlap:.2f}%"
    except ZeroDivisionError:
        return "Error: Probe diameter must not be 0."

# Function for Equation 2: Step size (nm)
def calculate_step_size(overlap_percent, probe_diameter):
    try:
        step_size = (1 - (overlap_percent / 100)) * probe_diameter
        return f"Step size: {step_size:.2f} nm"
    except ZeroDivisionError:
        return "Error: Probe diameter must not be 0."

# Function for Equation 3: Probe diameter (nm)
def calculate_probe_diameter_from_overlap(step_size, overlap_percent):
    try:
        probe_diameter = step_size / (1 - (overlap_percent / 100))
        return f"Probe diameter: {probe_diameter:.2f} nm"
    except ZeroDivisionError:
        return "Error: Overlap (%) must not be 100%."

# Function for Equation 4: Probe diameter from convergence angle
def calculate_probe_diameter_from_convergence(convergence_angle):
    try:
        ans = (convergence_angle / 1000) / (math.pi / 180)
        probe_diameter = 2 * math.tan(math.radians(ans)) * 100
        return f"Probe diameter: {probe_diameter:.2f} nm"
    except Exception as e:
        return f"Error: {e}"

# Interactive Widgets
# Widgets for Equations 1-3
step_size_slider = widgets.FloatSlider(value=10.0, min=0.1, max=100.0, step=0.1, description="Step size (nm):")
probe_diameter_slider = widgets.FloatSlider(value=50.0, min=0.1, max=200.0, step=0.1, description="Probe dia. (nm):")
overlap_percent_slider = widgets.FloatSlider(value=50.0, min=0.0, max=99.9, step=0.1, description="Overlap (%):")

# Widget for Equation 4
convergence_angle_slider = widgets.FloatSlider(value=25.0, min=0.1, max=50.0, step=0.1, description="Conv. angle (mrad):")

# Outputs
output_eq1 = Output()
output_eq2 = Output()
output_eq3 = Output()
output_eq4 = Output()

# Update Functions
def update_eq1(change):
    with output_eq1:
        output_eq1.clear_output()
        result = calculate_overlap_percent(step_size_slider.value, probe_diameter_slider.value)
        print(result)

def update_eq2(change):
    with output_eq2:
        output_eq2.clear_output()
        result = calculate_step_size(overlap_percent_slider.value, probe_diameter_slider.value)
        print(result)

def update_eq3(change):
    with output_eq3:
        output_eq3.clear_output()
        result = calculate_probe_diameter_from_overlap(step_size_slider.value, overlap_percent_slider.value)
        print(result)

def update_eq4(change):
    with output_eq4:
        output_eq4.clear_output()
        result = calculate_probe_diameter_from_convergence(convergence_angle_slider.value)
        print(result)

# Link widgets to updates
for widget in [step_size_slider, probe_diameter_slider]:
    widget.observe(update_eq1, names='value')
    widget.observe(update_eq3, names='value')

overlap_percent_slider.observe(update_eq2, names='value')
overlap_percent_slider.observe(update_eq3, names='value')

convergence_angle_slider.observe(update_eq4, names='value')

# Initial Calculation
update_eq1(None)
update_eq2(None)
update_eq3(None)
update_eq4(None)

# Display
display(VBox([
    Label("Calculating %overlap"),
    HBox([step_size_slider, probe_diameter_slider]),
    output_eq1,
    Label("Calculating step size"),
    HBox([overlap_percent_slider, probe_diameter_slider]),
    output_eq2,
    Label("Calculating probe diamter"),
    HBox([step_size_slider, overlap_percent_slider]),
    output_eq3,
    Label("Probe diamter from geometry"),
    convergence_angle_slider,
    output_eq4,
]))
